<a href="https://colab.research.google.com/github/mmaruthi/Deep_Learning_EVA4_Phase1/blob/master/S15-A/Depth_200K_to_300K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Mon May 11 08:37:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!git clone https://github.com/anilbhatt1/DenseDepth_V1.git

Cloning into 'DenseDepth_V1'...
remote: Enumerating objects: 238, done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 238
Receiving objects: 100% (238/238), 11.80 MiB | 35.11 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd gdrive/My Drive/DepthProject

/content/gdrive/My Drive/DepthProject


In [5]:
!ls -l

total 4
drwx------ 2 root root 4096 May 10 10:09 DenseDepth_V1


In [6]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O DenseDepth_V1/nyu.h5

--2020-05-11 08:39:06--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.88.67
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.88.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘DenseDepth_V1/nyu.h5’

DenseDepth_V1/nyu.h 100%[===================>] 164.89M  21.7MB/s    in 8.2s    

2020-05-11 08:39:15 (20.1 MB/s) - ‘DenseDepth_V1/nyu.h5’ saved [172897376/172897376]



In [7]:
ls DenseDepth_V1

augment.py      demo_rgb.npy      LICENSE       PyTorch/     utils.py
callbacks.py    DenseDepth.ipynb  loss.py       README.md
data.py         evaluate.py       model.py      Tensorflow/
demo_depth.npy  examples/         nyu.h5        test.py
demo.py         layers.py         __pycache__/  train.py


In [8]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from matplotlib import pyplot as plt
from DenseDepth_V1.layers import BilinearUpSampling2D
from DenseDepth_V1.utils import predict

MODEL_PATH = 'DenseDepth_V1/nyu.h5'

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(MODEL_PATH))

Using TensorFlow backend.


Loading model...

Model loaded (DenseDepth_V1/nyu.h5).


In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [10]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import skimage
from skimage.transform import resize
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from datetime import datetime 
import gc
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from functools import reduce
def make_batch(input_list, batch_size):
  def reducer(cumulator, item):
    if len(cumulator[-1]) < batch_size:
      cumulator[-1].append(item)
      return cumulator
    else:
      cumulator.append([item])
    return cumulator
  return reduce(reducer, input_list, [[]])

In [0]:
#!unzip -q "/content/gdrive/My Drive/EVA4P1_S15/FG_BG/FG_BG.zip"
FG_BG_zip =  ZipFile("/content/gdrive/My Drive/FG_BG.zip", 'r')

In [0]:
all_files_1 = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]

In [15]:
len(all_files_1)

800008

In [0]:
FG_BG_Images = all_files_1[400008:]

In [17]:
len(FG_BG_Images)

400000

In [0]:
def Depth_Creation(frm,to):
    global k, depth_log 
    # Depth Creation loop

    gc.collect()

    images    = []
    file_name = []
    for fg_bg_file in FG_BG_Images[frm:to]:
        imgdata = FG_BG_zip.read(fg_bg_file)
        img = Image.open(io.BytesIO(imgdata))
        img = img.resize((640, 480)) 
        x = np.clip(np.asarray(img, dtype=float) / 255, 0, 1)
        images.append(x)
        fg_bg_name = fg_bg_file.split('/')[-1].split('.')[0]  # We want to take "Img_fg_bg_184131" from 'G_BG/Img_fg_bg_184131.jpg'
        file_name.append(fg_bg_name)
    images = np.stack(images, axis=0)
    output = predict(model, images) 
    outputs = output.copy()

    gc.collect()
     
    # Depth Image saving loop
    for i in range(outputs.shape[0]):         
        op_img = outputs[i]
        op_img = resize(op_img, (200, 200))
        rescaled = op_img[:,:,0]
        rescaled = rescaled - np.min(rescaled)
        rescaled = rescaled / np.max(rescaled)
        depth    = Image.fromarray(rescaled * 255)
        depth    = depth.convert('L')
        depth_name  = file_name[i] + '_depth' + '.jpg'
        depth.save(f'{path_fg_bg_depth}{depth_name}')
        string = f'{depth_name},{file_name[i]}' + '\n'
        depth_log.write(string)
        k += 1

    # cleanup files
    del output, outputs, images

    # garbage collect
    gc.collect()
    return k   


In [0]:
x        = 0
delta    = 1000
iter_lst = []
for i in range(400):
    x1, y1 = x, x+delta
    tup = (x1,y1)
    iter_lst.append(tup)
    x = y1

In [20]:
iter_lst

[(0, 1000),
 (1000, 2000),
 (2000, 3000),
 (3000, 4000),
 (4000, 5000),
 (5000, 6000),
 (6000, 7000),
 (7000, 8000),
 (8000, 9000),
 (9000, 10000),
 (10000, 11000),
 (11000, 12000),
 (12000, 13000),
 (13000, 14000),
 (14000, 15000),
 (15000, 16000),
 (16000, 17000),
 (17000, 18000),
 (18000, 19000),
 (19000, 20000),
 (20000, 21000),
 (21000, 22000),
 (22000, 23000),
 (23000, 24000),
 (24000, 25000),
 (25000, 26000),
 (26000, 27000),
 (27000, 28000),
 (28000, 29000),
 (29000, 30000),
 (30000, 31000),
 (31000, 32000),
 (32000, 33000),
 (33000, 34000),
 (34000, 35000),
 (35000, 36000),
 (36000, 37000),
 (37000, 38000),
 (38000, 39000),
 (39000, 40000),
 (40000, 41000),
 (41000, 42000),
 (42000, 43000),
 (43000, 44000),
 (44000, 45000),
 (45000, 46000),
 (46000, 47000),
 (47000, 48000),
 (48000, 49000),
 (49000, 50000),
 (50000, 51000),
 (51000, 52000),
 (52000, 53000),
 (53000, 54000),
 (54000, 55000),
 (55000, 56000),
 (56000, 57000),
 (57000, 58000),
 (58000, 59000),
 (59000, 60000),
 (

In [0]:
iter_lst1 = iter_lst[200:300]

In [22]:
iter_lst1

[(200000, 201000),
 (201000, 202000),
 (202000, 203000),
 (203000, 204000),
 (204000, 205000),
 (205000, 206000),
 (206000, 207000),
 (207000, 208000),
 (208000, 209000),
 (209000, 210000),
 (210000, 211000),
 (211000, 212000),
 (212000, 213000),
 (213000, 214000),
 (214000, 215000),
 (215000, 216000),
 (216000, 217000),
 (217000, 218000),
 (218000, 219000),
 (219000, 220000),
 (220000, 221000),
 (221000, 222000),
 (222000, 223000),
 (223000, 224000),
 (224000, 225000),
 (225000, 226000),
 (226000, 227000),
 (227000, 228000),
 (228000, 229000),
 (229000, 230000),
 (230000, 231000),
 (231000, 232000),
 (232000, 233000),
 (233000, 234000),
 (234000, 235000),
 (235000, 236000),
 (236000, 237000),
 (237000, 238000),
 (238000, 239000),
 (239000, 240000),
 (240000, 241000),
 (241000, 242000),
 (242000, 243000),
 (243000, 244000),
 (244000, 245000),
 (245000, 246000),
 (246000, 247000),
 (247000, 248000),
 (248000, 249000),
 (249000, 250000),
 (250000, 251000),
 (251000, 252000),
 (252000, 25

In [0]:
k = 0
path_fg_bg_depth  = '/content/sample_data/ FG_BG_Depth_300K/'

In [25]:
len(iter_lst1)

100

In [26]:
for i in range(len(iter_lst1)):
    t1=time()
    frm, to           = iter_lst[i]
    depth_log         = open('/content/sample_data/ Log_Depth.txt', "a")
    k = Depth_Creation(frm,to)
    print('Depth Images created so far:',k, datetime.now())
    depth_log.close()
    t2=time()
    print(f'Model took {(t2-t1):.5f} s')


Depth Images created so far: 1000 2020-05-11 08:47:28.495152
Model took 262.13923 s
Depth Images created so far: 2000 2020-05-11 08:51:29.231350
Model took 240.73563 s
Depth Images created so far: 3000 2020-05-11 08:55:26.172835
Model took 236.94140 s
Depth Images created so far: 4000 2020-05-11 08:59:23.526300
Model took 237.35364 s
Depth Images created so far: 5000 2020-05-11 09:03:19.861389
Model took 236.33449 s
Depth Images created so far: 6000 2020-05-11 09:07:16.075953
Model took 236.21482 s
Depth Images created so far: 7000 2020-05-11 09:11:12.540282
Model took 236.46420 s
Depth Images created so far: 8000 2020-05-11 09:15:09.123128
Model took 236.58259 s
Depth Images created so far: 9000 2020-05-11 09:19:06.571325
Model took 237.44817 s
Depth Images created so far: 10000 2020-05-11 09:23:03.028663
Model took 236.45750 s
Depth Images created so far: 11000 2020-05-11 09:26:59.320565
Model took 236.29172 s
Depth Images created so far: 12000 2020-05-11 09:30:55.379511
Model took 2

In [27]:
!pwd

/content/gdrive/My Drive/DepthProject


In [0]:
os.chdir('/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Depth')

In [29]:
!pwd

/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Depth


In [0]:
zipname_fg_bg_depth = ' FG_BG_Depth_300K'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder(zipname_fg_bg_depth, '/content/sample_data/')